In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.append("..")

import numpy as np
import matplotlib.pyplot as plt
import os
from src import data
import json
from tqdm.auto import tqdm
from src.metrics import AggregateMetric

from src.utils.sweep_utils import read_sweep_results, relation_from_dict
import logging
from src.utils import logging_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format = logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout
)

In [2]:
############################################
sweep_root = "../results/sweep-24-trials"
model_name = "gptj"
############################################

sweep_path = f"{sweep_root}/{model_name}"

In [3]:
sweep_results = read_sweep_results(sweep_path, relation_names="country capital city")
list(sweep_results.keys())

2023-07-08 15:17:50 src.utils.sweep_utils DEBUG    depth=0 || --> ../results/sweep-24-trials/gptj
2023-07-08 15:17:50 src.utils.sweep_utils DEBUG    depth=1 ||     --> ../results/sweep-24-trials/gptj/person_occupation
2023-07-08 15:17:50 src.utils.sweep_utils DEBUG    depth=2 ||         --> ../results/sweep-24-trials/gptj/person_occupation/1_person_occupation_seed_71745
2023-07-08 15:17:50 src.utils.sweep_utils DEBUG    depth=3 ||             --> ../results/sweep-24-trials/gptj/person_occupation/1_person_occupation_seed_71745/args-20230629-173306.json
2023-07-08 15:17:50 src.utils.sweep_utils DEBUG    depth=3 ||             --> ../results/sweep-24-trials/gptj/person_occupation/1_person_occupation_seed_71745/results_all.json
2023-07-08 15:17:53 src.utils.sweep_utils DEBUG    depth=3 ||             --> ../results/sweep-24-trials/gptj/person_occupation/1_person_occupation_seed_71745/person_occupation.json
2023-07-08 15:17:55 src.utils.sweep_utils DEBUG    depth=2 ||         --> ../results

KeyboardInterrupt: 